### Load Data File

In [1]:
%matplotlib inline
import math
import matplotlib.pyplot as plt
from decimal import *
import numpy as np # linear algebra
import pandas as pd # read_csv and such
from io import StringIO # convert strings to buffers or something like that.
import os # for listing files in directory
import seaborn as sns
import scipy.stats as stats

pd.options.display.max_columns = 99
pd.options.display.max_rows = 99

In [2]:
flights_weather_path = '../data/flight_delays_2015/flights_weather.csv'
dtypes = {
    'ORIGIN_AIRPORT': 'str', 
    'DESTINATION_AIRPORT': 'str', 
    'IATA_CODE_x': 'str', 
    'origin_weather_station': 'str', 
    'IATA_CODE_y': 'str', 
    'destination_weather_station': 'str', 
    'OR_MAX': 'str', 
    'OR_MIN': 'str', 
    'OR_PRCP': 'str', 
    'DES_MAX': 'str', 
    'DES_MIN': 'str', 
    'DES_PRCP': 'str', 
    'OR_FRSHTT': 'str', 
    'DES_FRSHTT': 'str'
}
fw_df = pd.read_csv(flights_weather_path, dtype=dtypes, parse_dates=['DATE'])
fw_df = fw_df[['MONTH', 'DATE', 'ORIGIN_AIRPORT', 'WEATHER_DELAY', 'OR_TEMP', 'OR_VISIB', 'OR_WDSP', 'OR_MXSPD', 'OR_SNDP',
               'OR_PRCP', 'OR_GUST', 'OR_MAX', 'OR_MIN', 'OR_FOG', 'OR_RAIN_DRIZZLE', 'OR_SNOW_ICE_PELLETS',
               'OR_HAIL', 'OR_THUNDER', 'OR_TORNADO_FUNNEL_CLOUD', 'DES_TEMP', 'DES_VISIB', 'DES_WDSP',
               'DES_MXSPD', 'DES_SNDP', 'DES_PRCP', 'DES_GUST', 'DES_MAX', 'DES_MIN', 'DES_FOG', 'DES_RAIN_DRIZZLE',
               'DES_SNOW_ICE_PELLETS', 'DES_HAIL', 'DES_THUNDER', 'DES_TORNADO_FUNNEL_CLOUD']]
#fw_df = fw_df.dropna()
fw_df.head()

,MONTH,DATE,ORIGIN_AIRPORT,WEATHER_DELAY,OR_TEMP,OR_VISIB,OR_WDSP,OR_MXSPD,OR_SNDP,OR_PRCP,OR_GUST,OR_MAX,OR_MIN,OR_FOG,OR_RAIN_DRIZZLE,OR_SNOW_ICE_PELLETS,OR_HAIL,OR_THUNDER,OR_TORNADO_FUNNEL_CLOUD,DES_TEMP,DES_VISIB,DES_WDSP,DES_MXSPD,DES_SNDP,DES_PRCP,DES_GUST,DES_MAX,DES_MIN,DES_FOG,DES_RAIN_DRIZZLE,DES_SNOW_ICE_PELLETS,DES_HAIL,DES_THUNDER,DES_TORNADO_FUNNEL_CLOUD
0,1,2015-01-01,ANC,NaN,35.1,7.5,3.6,6.0,999.9,0.09G,999.9,43.0,32.0,True,True,False,False,False,False,32.9,10.0,4.6,11.1,999.9,0.00G,999.9,42.1,26.1,False,False,False,False,False,False
1,1,2015-01-01,ANC,NaN,35.1,7.5,3.6,6.0,999.9,0.09G,999.9,43.0,32.0,True,True,False,False,False,False,32.9,10.0,4.6,11.1,999.9,0.00G,999.9,42.1,26.1,False,False,False,False,False,False
2,1,2015-01-01,ANC,NaN,35.1,7.5,3.6,6.0,999.9,0.09G,999.9,43.0,32.0,True,True,False,False,False,False,32.9,10.0,4.6,11.1,999.9,0.00G,999.9,42.1,26.1,False,False,False,False,False,False
3,1,2015-01-01,ANC,NaN,35.1,7.5,3.6,6.0,999.9,0.09G,999.9,43.0,32.0,True,True,False,False,False,False,32.9,10.0,4.6,11.1,999.9,0.00G,999.9,42.1,26.1,False,False,False,False,False,False
4,1,2015-01-01,ANC,NaN,35.1,7.5,3.6,6.0,999.9,0.09G,999.9,43.0,32.0,True,True,False,False,False,False,32.9,10.0,4.6,11.1,999.9,0.00G,999.9,42.1,26.1,False,False,False,False,False,False


In [3]:
fw_df['OR_PRCP'] = pd.to_numeric(fw_df['OR_PRCP'].str.replace('A|B|C|D|E|F|G|H|I', ''))
fw_df['OR_MAX'] = pd.to_numeric(fw_df['OR_MAX'].str.replace('*', ''))
fw_df['OR_MIN'] = pd.to_numeric(fw_df['OR_MIN'].str.replace('*', ''))
fw_df.loc[fw_df['OR_TEMP'] == 9999.9, 'OR_TEMP'] = fw_df.OR_TEMP.mean()
fw_df.loc[fw_df['OR_WDSP'] == 999.9, 'OR_WDSP'] = fw_df.OR_WDSP.mean()
fw_df.loc[fw_df['OR_PRCP'] == 99.99, 'OR_PRCP'] = 0
fw_df.loc[fw_df['OR_VISIB'] == 999.9, 'OR_VISIB'] = fw_df.OR_VISIB.mean()
fw_df.loc[fw_df['OR_GUST'] == 999.9, 'OR_GUST'] = fw_df.OR_GUST.mean()
fw_df.loc[fw_df['OR_MAX'] == 9999.9, 'OR_MAX'] = fw_df.OR_MAX.mean()
fw_df.loc[fw_df['OR_MIN'] == 9999.9, 'OR_MIN'] = fw_df.OR_MIN.mean()
fw_df.loc[fw_df['OR_WDSP'] == 999.9, 'OR_WDSP'] = fw_df.OR_WDSP.mean()
fw_df.loc[fw_df['OR_MXSPD'] == 999.9, 'OR_MXSPD'] = fw_df.OR_MXSPD.mean()
fw_df.loc[fw_df['OR_SNDP'] == 999.9, 'OR_SNDP'] = 0
fw_df.loc[fw_df['OR_FOG'] == True, 'OR_FOGV'] = 1
fw_df.loc[fw_df['OR_FOG'] == False, 'OR_FOGV'] = 0
fw_df.loc[fw_df['OR_RAIN_DRIZZLE'] == True, 'OR_RAIN_DRIZZLEV'] = 1
fw_df.loc[fw_df['OR_RAIN_DRIZZLE'] == False, 'OR_RAIN_DRIZZLEV'] = 0
fw_df.loc[fw_df['OR_SNOW_ICE_PELLETS'] == True, 'OR_SNOW_ICE_PELLETSV'] = 1
fw_df.loc[fw_df['OR_SNOW_ICE_PELLETS'] == False, 'OR_SNOW_ICE_PELLETSV'] = 0
fw_df.loc[fw_df['OR_HAIL'] == True, 'OR_HAILV'] = 1
fw_df.loc[fw_df['OR_HAIL'] == False, 'OR_HAILV'] = 0
fw_df.loc[fw_df['OR_THUNDER'] == True, 'OR_THUNDERV'] = 1
fw_df.loc[fw_df['OR_THUNDER'] == False, 'OR_THUNDERV'] = 0
fw_df.loc[fw_df['OR_TORNADO_FUNNEL_CLOUD'] == True, 'OR_TORNADO_FUNNEL_CLOUDV'] = 1
fw_df.loc[fw_df['OR_TORNADO_FUNNEL_CLOUD'] == False, 'OR_TORNADO_FUNNEL_CLOUDV'] = 0
fw_df.loc[fw_df['WEATHER_DELAY'].isnull(), 'WEATHER_DELAY'] = 0


In [4]:
fw_df = fw_df[['MONTH', 'DATE', 'ORIGIN_AIRPORT', 'WEATHER_DELAY', 'OR_TEMP', 'OR_VISIB', 'OR_WDSP', 'OR_MXSPD', 'OR_SNDP',
               'OR_PRCP', 'OR_MAX', 'OR_MIN', 'OR_FOGV', 'OR_RAIN_DRIZZLEV', 'OR_SNOW_ICE_PELLETSV',
               'OR_HAILV', 'OR_THUNDERV', 'OR_TORNADO_FUNNEL_CLOUDV']]

In [5]:
fw_airports = fw_df.groupby(['DATE', 'ORIGIN_AIRPORT'], as_index=False).mean()

#### Train a SGD Regressor on Data

In [7]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(fw_df[['OR_TEMP', 'OR_VISIB',
               'OR_WDSP', 'OR_MXSPD', 'OR_SNDP', 'OR_PRCP','OR_MAX', 'OR_MIN', 'OR_FOGV',
               'OR_RAIN_DRIZZLEV', 'OR_SNOW_ICE_PELLETSV', 'OR_HAILV', 'OR_THUNDERV', 'OR_TORNADO_FUNNEL_CLOUDV']],
                                                    fw_df[['WEATHER_DELAY']])

/home/tcox/Envs/Volag/local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [23]:
from sklearn import linear_model
clf = linear_model.SGDRegressor()
clf.fit(X_train, y_train)

/home/tcox/Envs/Volag/local/lib/python2.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False)

In [24]:
clf.score(X_test, y_test)

-2.2088787633529505e+18

Based on this score, the model is the same as predicting the mean weather delay for all inputs (the negative score indicates that it is actually a little worse, but the value is essentially 0).

In [8]:
X_train, X_test, y_train, y_test = train_test_split(fw_df[['OR_TEMP', 'OR_VISIB',
               'OR_WDSP', 'OR_MXSPD', 'OR_SNDP', 'OR_PRCP','OR_MAX', 'OR_MIN']], fw_df[['WEATHER_DELAY']])

In [26]:
clf = linear_model.SGDRegressor()
clf.fit(X_train, y_train)

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False)

In [27]:
clf.score(X_test, y_test)

-8.9201568837612257e+19

Training with fewer variables results in a model that is essentially the same.

#### Ridge Regressor

In [28]:
from sklearn.linear_model import Ridge
clf = clf = Ridge(alpha=1.0)
clf.fit(X_train, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [29]:
clf.score(X_test, y_test)

0.0045940073950920812

The ridge regressor performs slightly better than estimating the mean, but not by much.


#### Elastic Lasso

In [9]:
X_train, X_test, y_train, y_test = train_test_split(fw_df[['OR_TEMP', 'OR_VISIB',
               'OR_WDSP', 'OR_SNDP', 'OR_PRCP']], fw_df[['WEATHER_DELAY']])

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression

X, y = make_regression(n_features=2, random_state=0)
regr = ElasticNet(random_state=0)
regr.fit(X_train, y_train)

In [ ]:
regr.score(X_test, y_test)

The Elastic Lass also performs about the same as predicting the mean for all inputs.

#### Support Vector Regressor

In [ ]:
from sklearn.svm import SVR
clf = clf = SVR(C=1.0, epsilon=0.2)
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

The support vector regressor is actually signifigantly worse than predicting the mean for all inputs.

